In [1]:
import scipy.io as io
Respdata=io.loadmat('./Main experiments/Mouse 1/RespData.mat')
# print(Respdata)

#### RespData

In [2]:
# show RespData
data = Respdata['RespData'][0][0][0][0][0][0]
print(data.shape)
print(data[:, :, 0, 0].shape)

(8, 720, 371, 5)
(8, 720)


In [ ]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torchvision.transforms import transforms
import torch.nn.functional as F 
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(tight_layout=True, figsize=(24, 24))
gs = gridspec.GridSpec(8, 2)

for i in range(7):
    ax = fig.add_subplot(gs[i, :])
    ax.plot(np.arange(720), data[i, :, 1, 0] - data[i+1, :, 1, 0])
    ax.set_ylabel('Response in Repeat %d' % (i+1))
plt.show()

In [ ]:
# a = np.array([[[1, 2, 4], [2, 5, 5]], [[6, 7, 3], [7, 4, 5]]])
# print(a)
# print(a.reshape((4, 3)))



In [ ]:
repeat, frame, cellNum = Respdata['RespData'][0][0][0][0][0][0][:, :, :, 0].shape

Day1_data = Respdata['RespData'][0][0][0][0][0][0][:, :, :, 0].reshape((repeat*frame, cellNum))
Day7_data = Respdata['RespData'][0][0][0][0][0][0][:, :, :, 1].reshape((repeat*frame, cellNum))
Day14_data = Respdata['RespData'][0][0][0][0][0][0][:, :, :, 2].reshape((repeat*frame, cellNum))
Day21_data = Respdata['RespData'][0][0][0][0][0][0][:, :, :, 3].reshape((repeat*frame, cellNum))
Day28_data = Respdata['RespData'][0][0][0][0][0][0][:, :, :, 4].reshape((repeat*frame, cellNum))


Day1_data = (Day1_data - np.mean(Day1_data, axis=0, keepdims=True)) / np.std(Day1_data, axis=0, keepdims=True)
Day7_data = (Day7_data - np.mean(Day7_data, axis=0, keepdims=True)) / np.std(Day7_data, axis=0, keepdims=True)
Day14_data = (Day14_data - np.mean(Day14_data, axis=0, keepdims=True)) / np.std(Day14_data, axis=0, keepdims=True)
Day21_data = (Day21_data - np.mean(Day21_data, axis=0, keepdims=True)) / np.std(Day21_data, axis=0, keepdims=True)
Day28_data = (Day28_data - np.mean(Day28_data, axis=0, keepdims=True)) / np.std(Day28_data, axis=0, keepdims=True)


print(len(Day1_data))
print(torch.tensor(Day1_data).shape)


In [ ]:
class fcn_autoencoder(nn.Module):
    
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(cellNum, 128),
            nn.ReLU(), 
            nn.Linear(128, 64), 
            nn.ReLU(), 
            nn.Linear(64, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12), 
            nn.ReLU(), 
            nn.Linear(12, 64), 
            nn.ReLU(),
            nn.Linear(64, 128), 
            nn.ReLU(), 
            nn.Linear(128, cellNum),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
class Day1Dataset(TensorDataset):

    def __init__(self, tensors):
        self.tensors = tensors
    
    def __getitem__(self, index):
        return self.tensors[index, :]

    def __len__(self):
        return len(self.tensors)

In [ ]:
# hyperparameters
num_epochs = 50000
batch_size = 64
learning_rate = 1e-3

# dataloader
x = torch.from_numpy(Day1_data)
day1TrainData = Day1Dataset(x)

train_sampler = RandomSampler(day1TrainData)
train_loader = DataLoader(day1TrainData, sampler=train_sampler, batch_size=batch_size)

# build model
Day1AutoEncoder = fcn_autoencoder().cuda()

# loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(Day1AutoEncoder.parameters(), lr=learning_rate)


In [ ]:
best_loss = np.inf
Loss = []

for epoch in range(num_epochs):
    total_loss = []
    for data in train_loader:

        responses = data.float().cuda()
        responses = responses.view(-1, cellNum)

        output = Day1AutoEncoder(responses)
        loss = criterion(output, responses)
        total_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss = np.mean(total_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(Day1AutoEncoder, 'best_model_Day1.pt')
    print('epoch %d, loss %f' % (epoch, mean_loss))
    Loss.append(mean_loss)
torch.save(Day1AutoEncoder, 'last_model_Day1.pt')

In [ ]:
plt.figure()
plt.plot(np.arange(num_epochs), np.array(Loss))
plt.show()


#### validation

In [ ]:
# MAPE 平均绝对百分比误差
# formula np.sum((y-yhat) ** 2 / y**2, axis=1) / cellNum
# 更换方法 np.sum((y-yhat) ** 2 / y**2, axis=1) / cellNum
def getMAPE(y, yHat):
    num = y.shape[1]
    return np.sum(np.abs(y-yHat) / np.abs(y), axis=1) / num

In [ ]:
# caculate MAPE and show frequency historgram
def signalScore(AE, dataSet):
    prediction = AE(dataSet)
    MAPE = getMAPE(dataSet.cpu().detach().numpy(), prediction.cpu().detach().numpy())
    df = pd.DataFrame(MAPE, columns=['MAPE'])
    print(df)
    df.MAPE.plot(kind='hist', label='MAPE', bins=20, range=(0, 10))
    plt.ylabel('Frequence')
    plt.xlabel('MAPE')
    plt.title('MAPE historgram')
    plt.legend()
    plt.show()


In [ ]:
model = torch.load('./best_model_Day1.pt')
signalScore(model, torch.from_numpy(Day1_data).float().cuda())

In [ ]:
def plotSignal(y, yHat):
    rowNum, num = y.shape
    print(rowNum)
    fig = plt.figure(tight_layout=True, figsize=(24, 24))
    gs = gridspec.GridSpec(rowNum, 2)

    for i in range(rowNum):
        ax = fig.add_subplot(gs[i, 0])
        ax.plot(np.arange(num), yHat[i, :], color='r')
        ax.plot(np.arange(num), y[i, :], color='b')
        ax = fig.add_subplot(gs[i, 1])
        ax.scatter(yHat[i, :], y[i, :])
    plt.show()



In [ ]:
plotSignal(Day1_data[:5, :], model.cpu()(torch.from_numpy(Day1_data[:5, :]).float()).detach().numpy())

In [ ]:
signalScore(model, torch.from_numpy(Day1_data).float())

In [ ]:
plotSignal(Day7_data[:5, :], model.cpu()(torch.from_numpy(Day7_data[:5, :]).float()).detach().numpy())

In [ ]:
# print(Day7_data)
signalScore(model, torch.from_numpy(Day7_data).float())

In [ ]:
plotSignal(Day14_data[:5, :], model.cpu()(torch.from_numpy(Day14_data[:5, :]).float()).detach().numpy())

In [ ]:
# print(Day14_data)
signalScore(model, torch.from_numpy(Day14_data).float())

In [ ]:
plotSignal(Day21_data[:5, :], model.cpu()(torch.from_numpy(Day21_data[:5, :]).float()).detach().numpy())

In [ ]:
signalScore(model, torch.from_numpy(Day21_data).float())

In [ ]:
plotSignal(Day28_data[:5, :], model.cpu()(torch.from_numpy(Day21_data[:5, :]).float()).detach().numpy())

In [ ]:
signalScore(model, torch.from_numpy(Day21_data).float())

#### StabilityData

In [ ]:
Stadata = io.loadmat('./Main experiments/Mouse 1/PDG_ChronicImaging_maps.mat')
print(Stadata)

In [ ]:
data = Stadata['PDG_ChronicImaging_maps'][0][0][0]
print(data.shape)